In [1]:
#数据分析模块
import pandas as pd
import numpy as np
import random as rnd
from datetime import datetime
import time
#可视化模块
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
plt.rcParams['font.sans-serif'] = ['SimHei'] # 显示中文标题 
plt.rcParams['axes.unicode_minus'] = False # 正常显示正负号 
%matplotlib inline


#模型训练模块
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
#导入数据
df = pd.read_excel('COMAP_RollerCoasterData_2019补充后.xlsx')

In [3]:
df.head()

,Name,Park,City/Region,City/State/Region,Country/Region,Geographic Region,Construction,Type,Status,Year/Date Opened,使用年长,Height (feet),Speed (mph),Length (feet),Inversions (YES or NO),Number of Inversions,Drop (feet),Duration (min:sec),G Force,Vertical Angle (degrees)
0,Montana Rusa,VulQano Park,Quito,Pichincha,Ecuador,South America,Steel,Sit Down,Operating,2005.0,14.0,36.1,NaN,1197.5,YES,1.0,NaN,NaN,NaN,NaN
1,Nemesis,Alton Towers,Alton,"Staffordshire, England",United Kingdom,Europe,Steel,Inverted,Operating,1994.0,25.0,42.6,50.0,2349.0,YES,4.0,104,01:20:00,3.5,NaN
2,Furius Baco,PortAventura Park,Salou,Tarragona,Spain,Europe,Steel,Wing,Operating,2007.0,12.0,NaN,83.9,2788.8,YES,1.0,NaN,00:55:00,4.7,NaN
3,Sidewinder,Elitch Gardens,Denver,Colorado,United States,North America,Steel,Sit Down,Operating,1995.0,24.0,56.0,45.0,635.0,YES,1.0,NaN,01:06:00,4,NaN
4,Doble Loop,Salitre Magico,Bogota,Cundinamarca,Colombia,South America,Steel,Sit Down,Operating,2004.0,15.0,59.1,56.0,2822.0,YES,2.0,NaN,NaN,NaN,NaN


In [4]:
# 拿要处理的列
new_df = df[['Construction','使用年长','Type','Height (feet)','Length (feet)','Number of Inversions','Drop (feet)','G Force','Vertical Angle (degrees)']]
# 速度这一列搞不了，单独处理了
sp = df.iloc[:,12:13]
new_df['Speed'] = sp.iloc[:,0:1]

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [5]:
# 处理时间，转换为秒
idx = df['Duration (min:sec)'][df['Duration (min:sec)'].isnull()==False].index
for i in idx:
    if i <=143:
        df['Duration (min:sec)'][i] = df['Duration (min:sec)'][i].hour*60 +  df['Duration (min:sec)'][i].minute
    if i > 144:
        df['Duration (min:sec)'][i] = df['Duration (min:sec)'][i].hour*60 +  df['Duration (min:sec)'][i].minute
new_df['Time'] = df['Duration (min:sec)']
new_df = new_df.drop([144])

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
new_df.head()

,Construction,使用年长,Type,Height (feet),Length (feet),Number of Inversions,Drop (feet),G Force,Vertical Angle (degrees),Speed,Time
0,Steel,14.0,Sit Down,36.1,1197.5,1.0,NaN,NaN,NaN,NaN,NaN
1,Steel,25.0,Inverted,42.6,2349.0,4.0,104,3.5,NaN,50.0,80
2,Steel,12.0,Wing,NaN,2788.8,1.0,NaN,4.7,NaN,83.9,55
3,Steel,24.0,Sit Down,56.0,635.0,1.0,NaN,4,NaN,45.0,66
4,Steel,15.0,Sit Down,59.1,2822.0,2.0,NaN,NaN,NaN,56.0,NaN


In [7]:
# 按材料进行分组
g = new_df.groupby('Construction')
w_df = g.get_group('Wood')
s_df = g.get_group('Steel')

In [8]:
w_df = g.get_group('Wood')
s_df = g.get_group('Steel')

In [9]:
w_df.shape

(50, 11)

In [10]:
s_df['Type'].unique() 

array(['Sit Down', 'Inverted', 'Wing', 'Steel', 'Stand Up', 'Flying',
       'Wood', 'Suspended'], dtype=object)

In [11]:
# 对乘坐类型进行数值转换
w_df['Type'] = w_df['Type'].map({'Sit Down' :0}).astype(float)
s_df['Type'] = s_df['Type'].map({'Sit Down':0, 'Inverted':1, 'Wing':2, 'Steel':3, 'Stand Up':4, 'Flying':5,
       'Wood':6, 'Suspended':7}).astype(float)

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [12]:
# 删除 Height (feet) 和 Length (feet)的空值行
idx = s_df['Height (feet)'][s_df['Height (feet)'].isnull()==True].index
s_df = s_df.drop(idx)
idx = s_df['Length (feet)'][s_df['Length (feet)'].isnull()==True].index
s_df = s_df.drop(idx)
idx = s_df['使用年长'][s_df['使用年长'].isnull()==True].index
s_df = s_df.drop(idx)
idx = s_df['Speed'][s_df['Speed'].isnull()==True].index
s_df = s_df.drop(idx)

idx = w_df['Height (feet)'][w_df['Height (feet)'].isnull()==True].index
w_df = w_df.drop(idx)
idx =  w_df['Length (feet)'][w_df['Length (feet)'].isnull()==True].index
w_df = w_df.drop(idx)
idx = w_df['使用年长'][w_df['使用年长'].isnull()==True].index
w_df = w_df.drop(idx)
idx = w_df['Speed'][w_df['Speed'].isnull()==True].index
w_df = w_df.drop(idx)

In [13]:
s_df.shape

(240, 11)

In [14]:
for i in range(0,240):
    print(str(i)+" : "+ str(s_df[['Drop (feet)']].values[i]))
print(s_df[['Drop (feet)']].values[192])
s_df = s_df.drop(s_df[['Drop (feet)']].index[192])
s_df['Drop (feet)'] = s_df[['Drop (feet)']].astype(float)

0 : [104]
1 : [nan]
2 : [nan]
3 : [nan]
4 : [nan]
5 : [nan]
6 : [nan]
7 : [nan]
8 : [nan]
9 : [nan]
10 : [98.4]
11 : [nan]
12 : [nan]
13 : [64]
14 : [nan]
15 : [nan]
16 : [nan]
17 : [nan]
18 : [80]
19 : [nan]
20 : [nan]
21 : [88.6]
22 : [nan]
23 : [105]
24 : [nan]
25 : [nan]
26 : [98]
27 : [nan]
28 : [nan]
29 : [nan]
30 : [nan]
31 : [nan]
32 : [nan]
33 : [90]
34 : [nan]
35 : [90]
36 : [nan]
37 : [90]
38 : [91]
39 : [90]
40 : [nan]
41 : [nan]
42 : [nan]
43 : [nan]
44 : [nan]
45 : [nan]
46 : [80]
47 : [100]
48 : [100]
49 : [nan]
50 : [81.2]
51 : [nan]
52 : [nan]
53 : [nan]
54 : [nan]
55 : [nan]
56 : [nan]
57 : [nan]
58 : [nan]
59 : [nan]
60 : [nan]
61 : [nan]
62 : [nan]
63 : [nan]
64 : [nan]
65 : [nan]
66 : [nan]
67 : [104.9]
68 : [nan]
69 : [nan]
70 : [nan]
71 : [nan]
72 : [nan]
73 : [nan]
74 : [nan]
75 : [nan]
76 : [nan]
77 : [nan]
78 : [nan]
79 : [nan]
80 : [nan]
81 : [nan]
82 : [nan]
83 : [nan]
84 : [nan]
85 : [nan]
86 : [111.8]
87 : [nan]
88 : [108]
89 : [54]
90 : [108]
91 : [128]
9

In [15]:
# 随机森林
def getNum_rfr(df,List,Type='float'):
    # 把已有的数值型特征取出来丢进Random Forest Regressor中
    feet_df = df[List]

    # 乘客分成已知年龄和未知年龄两部分
    known_feet = feet_df[feet_df[List[0]].notnull()].as_matrix()
    unknown_feet = feet_df[feet_df[List[0]].isnull()].as_matrix()

    # y即目标年龄
    y = known_feet[:, 0]

    # X即特征属性值
    X = known_feet[:, 1:]

    # fit到RandomForestRegressor之中
    rfr = RandomForestClassifier(n_estimators=100)

    # 导入归一化模块
    from sklearn.preprocessing import StandardScaler
    # 使用该类
    standarScaler = StandardScaler()
    # 传入训练数据作为标准
    standarScaler.fit(X)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # 获得归一化的训练数据
    X_train = standarScaler.transform(X)
    # 获得归一化的测试数据
    X_test_standar = standarScaler.transform(unknown_feet[:, 1:])

    # 只能int类型！！!
    rfr.fit(X_train, y.astype(Type))

    # 用得到的模型进行未知年龄结果预测
    predictedAges = rfr.predict(X_test_standar)
    #  print predictedAges
    # 用得到的预测结果填补原缺失数据
    df.loc[ (df[List[0]].isnull()), List[0]] = predictedAges
    return df

# 决策树
def getNum(df,List,Type='float'):
    feet_df = df[List]

    # 乘客分成已知年龄和未知年龄两部分
    known_feet = feet_df[feet_df[List[0]].notnull()].as_matrix()
    unknown_feet = feet_df[feet_df[List[0]].isnull()].as_matrix()

    # y即目标年龄
    y = known_feet[:, 0]

    # X即特征属性值
    X = known_feet[:, 1:]


    # 导入归一化模块
    from sklearn.preprocessing import StandardScaler
    # 使用该类
    standarScaler = StandardScaler()
    # 传入训练数据作为标准
    standarScaler.fit(X)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # 获得归一化的训练数据
    X_train = standarScaler.transform(X)
    # 获得归一化的测试数据
    X_test_standar = standarScaler.transform(unknown_feet[:, 1:])

    # fit到RandomForestRegressor之中
    decision_tree = DecisionTreeClassifier()

    decision_tree.fit(X_train, y.astype(Type))

    # 用得到的模型进行未知年龄结果预测
    predictedAges = decision_tree.predict(X_test_standar)
    #  print predictedAges
    # 用得到的预测结果填补原缺失数据
    df.loc[ (df[List[0]].isnull()), List[0]] = predictedAges
    return df

In [16]:
List = ['Drop (feet)','使用年长', 'Type', 'Height (feet)', 'Length (feet)']
s_df = getNum_rfr(s_df,List,Type='int')

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [17]:
List = ['Time','使用年长', 'Type', 'Height (feet)', 'Length (feet)']
s_df = getNum(s_df,List)

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [18]:
List = ['Vertical Angle (degrees)','使用年长', 'Type', 'Height (feet)', 'Length (feet)','Time','Speed','Drop (feet)']
s_df = getNum_rfr(s_df,List,Type='int')

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [19]:
# 删除空格
for i in range(0,239):
    print(str(i)+" : "+ str(s_df[['G Force']].values[i]))
print(s_df[['G Force']].values[232])
s_df = s_df.drop(s_df[['G Force']].index[232])
s_df['G Force'] = s_df[['G Force']].astype(float)

0 : [3.5]
1 : [4]
2 : [nan]
3 : [nan]
4 : [nan]
5 : [nan]
6 : [nan]
7 : [nan]
8 : [nan]
9 : [nan]
10 : [4.5]
11 : [4.5]
12 : [4.5]
13 : [nan]
14 : [nan]
15 : [nan]
16 : [4.5]
17 : [5]
18 : [nan]
19 : [nan]
20 : [nan]
21 : [4]
22 : [nan]
23 : [nan]
24 : [4.5]
25 : [nan]
26 : [3]
27 : [nan]
28 : [nan]
29 : [nan]
30 : [3.8]
31 : [nan]
32 : [nan]
33 : [nan]
34 : [nan]
35 : [nan]
36 : [nan]
37 : [nan]
38 : [nan]
39 : [nan]
40 : [4.9]
41 : [4]
42 : [4]
43 : [4]
44 : [nan]
45 : [nan]
46 : [nan]
47 : [nan]
48 : [nan]
49 : [nan]
50 : [4.6]
51 : [nan]
52 : [nan]
53 : [nan]
54 : [nan]
55 : [nan]
56 : [nan]
57 : [nan]
58 : [nan]
59 : [nan]
60 : [nan]
61 : [nan]
62 : [nan]
63 : [nan]
64 : [nan]
65 : [nan]
66 : [nan]
67 : [4]
68 : [4.7]
69 : [4.9]
70 : [nan]
71 : [4.3]
72 : [4.3]
73 : [nan]
74 : [nan]
75 : [5.2]
76 : [5.2]
77 : [5.2]
78 : [5.2]
79 : [5.2]
80 : [5.2]
81 : [5.2]
82 : [5.2]
83 : [5.2]
84 : [5.2]
85 : [5.2]
86 : [4.7]
87 : [nan]
88 : [nan]
89 : [nan]
90 : [nan]
91 : [nan]
92 : [nan]
93 

In [20]:
List = ['G Force','使用年长', 'Type', 'Height (feet)', 'Length (feet)','Time','Speed','Drop (feet)']
s_df = getNum(s_df,List,Type='int')

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [21]:
List = ['Drop (feet)','使用年长', 'Type', 'Height (feet)', 'Length (feet)']
w_df = getNum_rfr(w_df,List,Type='int')
List = ['Time','使用年长', 'Type', 'Height (feet)', 'Length (feet)']
w_df = getNum(w_df,List)

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-pa

In [22]:
List = ['Vertical Angle (degrees)','使用年长', 'Type', 'Height (feet)', 'Length (feet)','Time','Speed','Drop (feet)']
w_df = getNum(w_df,List)

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [23]:
List = ['G Force','使用年长', 'Type', 'Height (feet)', 'Length (feet)','Time','Speed','Drop (feet)']
w_df = getNum_rfr(w_df,List,Type='int')

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [24]:
w_df

,Construction,使用年长,Type,Height (feet),Length (feet),Number of Inversions,Drop (feet),G Force,Vertical Angle (degrees),Speed,Time
22,Wood,2.0,0.0,83.0,2290.0,1.0,80.5,2,65.0,45.8,105
55,Wood,6.0,0.0,107.0,2937.0,3.0,162,3,81.0,68.0,87
108,Wood,14.0,0.0,136.0,4746.0,1.0,140,3,65.0,60.0,154
138,Wood,5.0,0.0,165.0,3100.0,2.0,180,3,85.0,72.0,205
146,Wood,3.0,0.0,183.8,4150.3,3.0,160.8,4,83.0,71.5,120
177,Wood,84.0,0.0,55.0,2650.0,0.0,65,2,50.0,50.0,105
185,Wood,95.0,0.0,70.0,2640.0,0.0,65,2,50.0,55.0,112
186,Wood,39.0,0.0,71.0,2670.0,0.0,65,2,50.0,45.0,90
190,Wood,94.0,0.0,73.0,2600.0,0.0,65,2,50.0,55.0,112
192,Wood,16.0,0.0,75.0,2600.0,0.0,72,2,53.0,50.0,105


In [25]:
w_df.to_csv('MuTou.csv')

In [26]:
s_df.to_csv('TieLu.csv')

#### 到此数据补充完毕

In [27]:
St_s = w_df[['使用年长','Type','Height (feet)','Length (feet)','Number of Inversions','Drop (feet)','G Force','Vertical Angle (degrees)','Speed','Time']]

In [32]:
import numpy as np
x=np.array([0.6,0.67,0.6]);
ca=np.array([0.1,0.335,0.1]);

In [33]:
ca1 = x-ca / math.sqrt(-math.log(0.1))

In [34]:
ca1

array([0.53409898, 0.44923157, 0.53409898])

In [35]:
stand_sdf = s_df[['Height (feet)','Speed','Length (feet)','Number of Inversions','Drop (feet)','Time','Vertical Angle (degrees)','G Force']]

In [36]:
def getStand(df):
    m = np.array(df.min().values)
    M = np.array(df.max().values)
    fenZi = M - m
    df = (df - m)/fenZi
    return df

In [37]:
stand_sdf  = getStand(stand_sdf)

In [38]:
stand_sdf

,Height (feet),Speed,Length (feet),Number of Inversions,Drop (feet),Time,Vertical Angle (degrees),G Force
1,0.031950,0.181668,0.295282,0.285714,0.196931,0.232143,0.328947,0.227273
3,0.063328,0.140380,0.058115,0.071429,0.391304,0.169643,0.328947,0.454545
4,0.070588,0.231214,0.360731,0.142857,0.010230,0.285714,0.013158,0.000000
5,0.079720,0.140380,0.215857,0.071429,0.181586,0.209821,0.684211,0.000000
6,0.082062,0.076796,0.129141,0.142857,0.391304,0.209821,0.552632,0.909091
7,0.082062,0.076796,0.129141,0.142857,0.391304,0.209821,0.552632,0.909091
8,0.082062,0.076796,0.129141,0.142857,0.094629,0.142857,0.118421,0.454545
9,0.082062,0.076796,0.129141,0.142857,0.094629,0.142857,0.118421,0.454545
10,0.082062,0.076796,0.129099,0.142857,0.391304,0.209821,0.552632,0.909091
12,0.100795,0.140380,0.115539,0.214286,0.181586,0.209821,0.684211,0.454545


In [39]:
#导入数据
stand = pd.read_excel('数值型标准.xlsx',index_col=1)

In [40]:
stand

,Unnamed: 0,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
过山车指标,NaN,惊悚型,NaN,刺激型,NaN,家庭型,NaN,儿童型,NaN
Height (feet),NaN,16.4,456.04,16.4,180.12,5.91,93.5,4.27,29.86
Speed (mph),NaN,61.18,386.4,28.98,156.17,11.592,121.716,9.66,56.189
Length (feet),NaN,319.88,8133.2,170.6,11811,180.12,3044.62,75.46,711.94
Number of Inversions,NaN,0,14,0,1,0,1,0,0
Drop (feet),NaN,9.5,127,2.1,38,2.4,20,1.8,2.4
Duration (min:sec),NaN,00:25:00,04:12:00,00:38:00,06:30:00,00:30:00,04:10:00,00:15:00,01:50:00


In [41]:
stand = stand.T.iloc[1:,4:]
# 处理时间
for i in range(0,8):
    stand['Duration (min:sec)'][i] = stand['Duration (min:sec)'][i].hour*60 +  stand['Duration (min:sec)'][i].minute
stand = stand.astype(float)

In [42]:
stand = stand.iloc[0:9,:]

In [43]:
stand  = getStand(stand) 

In [44]:
# 计算贝塔
a1=np.array(stand.iloc[0,:].values);
b1=np.array(stand.iloc[1,:].values);
beiTa = (b1-a1)/2

In [45]:
beiTa

array([0.48657503, 0.43162393, 0.3328908 , 0.5       , 0.4692492 ,
       0.30266667, 0.35185185, 0.5       ])

In [46]:
 stand.index = ['惊悚型下限','惊悚型上限','刺激型上限','刺激型下限','家庭型上限','家庭型下限','儿童型上限','儿童型下限']

In [47]:
stand

,Height (feet),Speed (mph),Length (feet),Number of Inversions,Drop (feet),Duration (min:sec),Vertical Angle (degrees),G
惊悚型下限,0.026850,0.136752,0.020827,0.000000,0.061502,0.026667,0.296296,0.0
惊悚型上限,1.000000,1.000000,0.686609,1.000000,1.000000,0.632000,1.000000,1.0
刺激型上限,0.026850,0.051282,0.008107,0.000000,0.002396,0.061333,0.164609,0.0
刺激型下限,0.389247,0.388889,1.000000,0.071429,0.289137,1.000000,0.716049,1.0
家庭型上限,0.003630,0.005128,0.008918,0.000000,0.004792,0.040000,0.205761,0.0
家庭型下限,0.197512,0.297436,0.253005,0.071429,0.145367,0.626667,0.469136,1.0
儿童型上限,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
儿童型下限,0.056644,0.123504,0.054235,0.000000,0.004792,0.253333,0.000000,1.0


In [48]:
stand.T.to_csv('标准化2.csv',index=0)

In [49]:
x = stand.iloc[6,:].values

In [50]:
W = -(x-beiTa)/ math.sqrt(-math.log(0.1))

In [51]:
W

array([0.32065792, 0.28444459, 0.21937844, 0.32950511, 0.30924002,
       0.19946043, 0.23187397, 0.32950511])

In [52]:
stand.iloc[6,:]

Height (feet)               0.0
 Speed (mph)                0.0
Length (feet)               0.0
Number of Inversions        0.0
Drop (feet)                 0.0
Duration (min:sec)          0.0
Vertical Angle (degrees)    0.0
G                           0.0
Name: 儿童型上限, dtype: float64

In [53]:
def getFen(df):
    df_fen = df*W
    return df_fen

In [54]:
df_fen = getFen(stand_sdf)

In [55]:
df_fen

,Height (feet),Speed,Length (feet),Number of Inversions,Drop (feet),Time,Vertical Angle (degrees),G Force
1,0.010245,0.051674,0.064778,0.094144,0.060899,0.0463033,0.076274,0.074888
3,0.020307,0.039930,0.012749,0.023536,0.121007,0.033837,0.076274,0.149775
4,0.022634,0.065768,0.079137,0.047072,0.003164,0.0569887,0.003051,0.000000
5,0.025563,0.039930,0.047354,0.023536,0.056154,0.0418511,0.158651,0.000000
6,0.026314,0.021844,0.028331,0.047072,0.121007,0.0418511,0.128141,0.299550
7,0.026314,0.021844,0.028331,0.047072,0.121007,0.0418511,0.128141,0.299550
8,0.026314,0.021844,0.028331,0.047072,0.029263,0.0284943,0.027459,0.149775
9,0.026314,0.021844,0.028331,0.047072,0.029263,0.0284943,0.027459,0.149775
10,0.026314,0.021844,0.028322,0.047072,0.121007,0.0418511,0.128141,0.299550
12,0.032321,0.039930,0.025347,0.070608,0.056154,0.0418511,0.158651,0.149775


In [56]:
List = []
for i in range(0,238):
    t = 0
    for x in range(0,8):
        t += df_fen.iloc[i,x]
    List.append(t)
df_fen['Sum'] = List

In [57]:
df_fen.sort_values('Sum',ascending=False)

,Height (feet),Speed,Length (feet),Number of Inversions,Drop (feet),Time,Vertical Angle (degrees),G Force,Sum
299,0.320658,0.234884,0.088122,0.000000,0.309240,0,0.137294,0.299550,1.389748
295,0.254055,0.196833,0.081116,0.000000,0.295004,0.0017809,0.137294,0.299550,1.265632
167,0.167929,0.141400,0.137880,0.023536,0.191397,0.19946,0.122039,0.269595,1.253237
294,0.222293,0.157372,0.193880,0.000000,0.231732,0.160281,0.109835,0.149775,1.225168
158,0.132188,0.108047,0.167714,0.094144,0.136825,0.108635,0.137294,0.299550,1.184396
298,0.293626,0.216093,0.078469,0.000000,0.295004,0.0017809,0.137294,0.149775,1.172042
291,0.208026,0.150326,0.160003,0.000000,0.220660,0.160281,0.106784,0.149775,1.155855
292,0.211030,0.152675,0.193667,0.000000,0.215914,0.0997302,0.106784,0.149775,1.129576
164,0.148256,0.118147,0.098045,0.070608,0.147897,0.0747977,0.137294,0.299550,1.094595
297,0.289872,0.169117,0.030963,0.000000,0.238139,0,0.137294,0.224663,1.090046


In [58]:
idx = df_fen.sort_values('Sum',ascending=False).index[0:10]

In [59]:
df.iloc[idx,:1]

,Name
299,Kingda Ka
295,Red Force
167,Hyperion
294,Fury 325
158,Steel Vengeance
298,Top Thrill Dragster
291,Leviathan
292,Millennium Force
164,Dinoconda
297,Superman: Escape from Krypton


In [60]:
df_fen = df_fen.sort_values('Sum',ascending=False)
idx = [i for i in range(1,239)]
df_fen['Rank'] = idx

In [61]:
df_fen['Name'] = df.iloc[df_fen.index,:1]

In [62]:
df_fen

,Height (feet),Speed,Length (feet),Number of Inversions,Drop (feet),Time,Vertical Angle (degrees),G Force,Sum,Rank,Name
299,0.320658,0.234884,0.088122,0.000000,0.309240,0,0.137294,0.299550,1.389748,1,Kingda Ka
295,0.254055,0.196833,0.081116,0.000000,0.295004,0.0017809,0.137294,0.299550,1.265632,2,Red Force
167,0.167929,0.141400,0.137880,0.023536,0.191397,0.19946,0.122039,0.269595,1.253237,3,Hyperion
294,0.222293,0.157372,0.193880,0.000000,0.231732,0.160281,0.109835,0.149775,1.225168,4,Fury 325
158,0.132188,0.108047,0.167714,0.094144,0.136825,0.108635,0.137294,0.299550,1.184396,5,Steel Vengeance
298,0.293626,0.216093,0.078469,0.000000,0.295004,0.0017809,0.137294,0.149775,1.172042,6,Top Thrill Dragster
291,0.208026,0.150326,0.160003,0.000000,0.220660,0.160281,0.106784,0.149775,1.155855,7,Leviathan
292,0.211030,0.152675,0.193667,0.000000,0.215914,0.0997302,0.106784,0.149775,1.129576,8,Millennium Force
164,0.148256,0.118147,0.098045,0.070608,0.147897,0.0747977,0.137294,0.299550,1.094595,9,Dinoconda
297,0.289872,0.169117,0.030963,0.000000,0.238139,0,0.137294,0.224663,1.090046,10,Superman: Escape from Krypton


In [63]:
pd.merge(s_df,w_df,on)

ValueError: You are trying to merge on float64 and object columns. If you wish to proceed you should use pd.concat